# Libraries

In [1]:
import numpy as np
import pandas as pd
import tflearn, h5py, cv2
import matplotlib.pyplot as plt

import sys
sys.path.append('..')
from src.models.cnn_model import CNNModel

/home/thuync/.virtualenvs/lungcancer/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/thuync/.virtualenvs/lungcancer/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/thuync/.virtualenvs/lungcancer/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/thuync/.virtualenvs/lungcancer/lib

Instructions for updating:
Colocations handled automatically by placer.


# Build components

In [2]:
# Dataset

h5f = h5py.File('../src/data/train.h5', 'r')
X_train_images = h5f['X']
Y_train_labels = h5f['Y']

print("X_train_images", X_train_images.shape)
print("Y_train_labels", Y_train_labels.shape)

X_train_images (5187, 50, 50, 1)
Y_train_labels (5187, 2)


In [3]:
# Model

convnet  = CNNModel()
network = convnet.define_network(X_train_images)
model = tflearn.DNN(network, tensorboard_verbose=0, checkpoint_path="../ckpt/nodule3-classifier.tfl.ckpt")
model.load("../ckpt/nodule3-classifier.tfl")

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /home/thuync/thuync/LungCancerDetection/ckpt/nodule3-classifier.tfl


# Get clusters

In [4]:
# Inference

preds = model.predict(X_train_images[:,:,:,:])
print("preds", preds.shape)

preds (5187, 2)


In [5]:
# Get clusters

pos_indicator = Y_train_labels[:,1]==1
neg_indicator = Y_train_labels[:,0]==1

pos_embeddings = preds[pos_indicator,:]
neg_embeddings = preds[neg_indicator,:]
print(pos_embeddings.shape, neg_embeddings.shape)

avg_pos_embedding = pos_embeddings.mean(axis=0)
avg_neg_embedding = neg_embeddings.mean(axis=0)
print("avg_pos_embedding", avg_pos_embedding)
print("avg_neg_embedding", avg_neg_embedding)

(845, 2) (4342, 2)
avg_pos_embedding [ 0.00042632 -0.03634213]
avg_neg_embedding [ 0.00042632 -0.0363417 ]


# Testing

In [7]:
# Dataset

h5f2 = h5py.File('../src/data/test.h5', 'r')
X_test_images = h5f2['X']
Y_test_labels = h5f2['Y']

print("X_test_images", X_test_images.shape)
print("Y_test_labels", Y_test_labels.shape)

X_test_images (1622, 50, 50, 1)
Y_test_labels (1622, 2)


In [8]:
# Inference

embeddings = model.predict(X_test_images[:,:,:,:])
print("embeddings", embeddings.shape)

embeddings (1622, 2)


In [17]:
# Cluster

pos_dists = ((embeddings-avg_pos_embedding)**2).sum(axis=1)[:,np.newaxis]
neg_dists = ((embeddings-avg_neg_embedding)**2).sum(axis=1)[:,np.newaxis]
dists = np.hstack([neg_dists, pos_dists])
pred_indicies = np.argmin(dists, axis=1)

print("neg_ratio:", (pred_indicies==0).sum())
print("pos_ratio:", (pred_indicies==1).sum())

neg_ratio: 1622
pos_ratio: 0
